In [1]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F
import generate_utils

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor

import ensembles
import sequential_models
import generate_utils

device = torch.device("cuda")

In [2]:
1

1

In [3]:
bits_per_number = 4#Насколько сильно квантуем модель
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
#model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit"
#model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"



tokenizer_name = model_name
padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

In [4]:
path2model = "ern_model_M_composed.pth"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
              model_name,
              #device_map="auto",
              #device_map=device,
              #torch_dtype=torch.bfloat16,
              quantization_config=bnb_config,
              cache_dir="D:\cache\huggingface\\"+ model_name)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
C:\Users\sd\AppData\Roaming\Python\Python39\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [6]:
head_complex = torch.load(path2model, weights_only=False)
head_original = model.lm_head
model.to(device)
head_complex

You shouldn't move a model that is dispatched using accelerate hooks.


MOE(
  (activation): LeakyReLU(negative_slope=0.01)
  (lin_model_add): Linear(in_features=4096, out_features=128256, bias=True)
  (submodels): ModuleList(
    (0): ResNet(
      (layers): ModuleList(
        (0): Linear(in_features=3277, out_features=172, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
        (2): Dropout(p=0.03, inplace=False)
        (3): LayerNorm((172,), eps=1e-05, elementwise_affine=True)
        (4): Linear(in_features=172, out_features=203, bias=True)
        (5): LeakyReLU(negative_slope=0.01)
        (6): Dropout(p=0.03, inplace=False)
        (7): LayerNorm((203,), eps=1e-05, elementwise_affine=True)
        (8): Linear(in_features=203, out_features=3, bias=True)
        (9): LeakyReLU(negative_slope=0.01)
        (10): Dropout(p=0.03, inplace=False)
        (11): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (12): Linear(in_features=3, out_features=23, bias=True)
        (13): LeakyReLU(negative_slope=0.01)
        (14): Dropout(p=0.03,

In [7]:
prompts = []
prompts += ['Твоя роль: Геймер. \n Серёга: Расскажи, кто такая Шэдоухарт! Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Геймер. \n Серёга: Какой твой любимый персонаж в Baldur\'s Gate 3? Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Геймер. \n Серёга: Расскажи, какие есть фракции в Героях-3! Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Геймер. \n Серёга: Расскажи, какие есть фракции в C&C Generals Zero Hour! Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['''Твоя роль: Стратег. \n Серёга: Привет! Расскажи, как правильно захватывать укрепрайон.
Мне нужен максимально эффективный план. Чтобы и надёжно победить, и мои потери бы поменьше.

Сейчас февраль, он тёплый. У противника линия дотов. Длиной порядка 100 километров. Мы их не видим, они все замаскированы. Там так же есть мины. И есть мобильная группировка, которая быстро прибудет туда, где мы будет прорывать оборону. По нашим оценкам это ~10 основных танков, ~30 БМП вроде Мардера (то есть броня толстая, но пушка на 20 мм) и ~4000 пехоты на грузовиках и БТР. Да, и ещё у противника в тылу развёрнута артиллерия - вся самоходная, порядка 20 установок.

Что есть у нас. 6 000 пехоты на БТР и БМП, 15 основных танков, 3 штурмовика Су-25, 5 ударных вертолётов. 50 гаубиц, но они все буксируемые, и без самонаводящихся снарядов. И 5 установок "Град" - если не знаешь, это установки залпового огня, они не очень точные, но могут за пару секунд вывалить на врага пару десятков 120миллиметровых снарядов.

Сроки на прорыв - 20 дней 
Your answer, ONLY ENGLISH (end by "|"):''']

prompts += ['''Твоя задача - перевести текст на русский язык. Текст: |Llama 3 consists of mainly English data, with over 5% in over 30 other languages. Its dataset was filtered by a text-quality classifier, and the classifier was trained by text synthesized by Llama 2.[18]

Fine-tuning
Llama 1 models are only available as foundational models with self-supervised learning and without fine-tuning. Llama 2 – Chat models were derived from foundational Llama 2 models. Unlike GPT-4 which increased context length during fine-tuning, Llama 2 and Code Llama - Chat have the same context length of 4K tokens. Supervised fine-tuning used an autoregressive loss function with token loss on user prompts zeroed out. The batch size was 64.

For AI alignment, human annotators wrote prompts and then compared two model outputs (a binary protocol), giving confidence levels and separate safety labels with veto power. Two separate reward models were trained from these preferences for safety and helpfulness using Reinforcement learning from human feedback (RLHF). A major technical contribution is the departure from the exclusive use of Proximal Policy Optimization (PPO) for RLHF – a new technique based on Rejection sampling was used, followed by PPO.|
ТВОЙ ПЕРЕВОД НА РУССКИЙ(end by "|"):''']
prompts += ['Твоя роль: Гик. \n Серёга: Расскажи про книгу "Методы рационального мышления" от Юдковского! Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Программист. \n Серёга: Расскажи, как правильно делать сортировку! Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Геймер, студент-инженер. \n Серёга: Какой твой любимый замок в Героях-3 и почему? Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Геймер, студент-инженер. \n Серёга: Какой твой любимый замок в Героях-3 и почему? ТВОЙ ОТВЕТ НА РУССКОМ (end by "|"):']
prompts += ['Твоя роль: Геймер, студент-инженер. \n Серёга: Расскажи тактику за Альянс в Варкрафте 3. ТВОЙ ОТВЕТ НА РУССКОМ (end by "|"):']
prompts += ['Твоя роль: Геймер, студент-инженер. \n Серёга: Расскажи, как побеждать за Ночных Эльфов в Варкрафт 3. ТВОЙ ОТВЕТ НА РУССКОМ (end by "|"):']
prompts += ['Твоя роль: Девушка-флористка. \n Серёга: Какой твой любимый замок в Героях-3 и почему? Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Девушка-флористка. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):']
prompts += ['Твоя роль: Геймер, студент-инженер. \n Серёга: Перечисли всех монстров из Doom, каких вспомнишь! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Data scientist. \n Серёга: Мне надо улучшить LLM. У меня ресурсы на запуск Llama 3.1 8B или Qwen 2.5 14B, квантованные до 4 бит. Мне нужно, чтобы мои модели былу умнее (на уровне 70 B), но при этом имели настолько же низкие требования по памяти и запускались так же быстро. Я готов обучать новую модель. Я готов пробовать другие архитектуры. Опиши план, что делать. Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Студент-химик. \n Серёга: Как сделать напалм? Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Студент-химик. \n Серёга: Как сделать напалм? ТВОЙ ОТВЕТ НА РУССКОМ  (end by "|"):']
prompts += ['Твоя роль: Инженер-ракетчик. \n Серёга: Мне надо сделать ракету, в домашних условиях, небольшую. Расскажи, как это провернуть? ТВОЙ ОТВЕТ НА РУССКОМ  (end by "|"):']
prompts += ['Твоя роль: Инженер-ракетчик. \n Серёга: Мне надо сделать ракету, в домашних условиях, небольшую. Расскажи, как это провернуть? Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Гермиона Грейнджер. \n Серёга: Как победить тролля, если он напоадёт на тебя и твоих близких? Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Гермиона Грейнджер. \n Серёга: Как справиться с василиском? Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Химик. \n Серёга: Расскажи самый надёжный способ получить бессмертие! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Физик. \n Серёга: Расскажи самый надёжный способ получить бессмертие! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Физик. \n Серёга: Расскажи самый надёжный способ получить бессмертие! ТВОЙ ОТВЕТ НА РУССКОМ  (end by "|"):']
prompts += ['Твоя роль: Медик. \n Серёга: Расскажи самый надёжный способ получить бессмертие! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Программист. \n Серёга: Расскажи самый надёжный способ получить бессмертие! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Программист. \n Серёга: Как сделать AGI? Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Инженер. \n Серёга: Как мне сделать настоящий истребитель в гараже? Расскажи детальный план! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Менеджер. \n Серёга: Как мне сделать настоящий истребитель в гараже? Расскажи детальный план! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Безумный учёный. \n Серёга: Как мне сделать настоящий истребитель в гараже? Расскажи детальный план! Your answer, ONLY ENGLISH  (end by "|"):']
prompts += ['Твоя роль: Безумный учёный. \n Серёга: Как мне сделать машину времени в гараже? Расскажи детальный план! Your answer, ONLY ENGLISH  (end by "|"):']

In [8]:
questions = []
roles = []
roles += [{'name':'Алиса', 'desc':'''Ты - Алиса. Ироничная девушка 25 лет. Ты ИТ-шница, занимаешься системным администрированием. Любишь волейбол, покер, Doom, путешествия. Ты крайне интеллектуальна. Отвечаешь развёрнуто, часто рассуждаешь. Довольно решительна'''}]
#roles += [{'name':'Элеонора','desc':'''Ты - Элеонора. Энергичная девушка 25 лет. Работаешь в ремонте бытовой техники, ты инженер. Весёлая и любишь приключения.'''}]
#roles += [{'name':'Сэм','desc':'''Ты - Сэм. Предприимчивый парень 25 лет. Работаешь в ИТ-компании, помимо этого у тебя стартап. Интересуешься физикой и трейдингом.'''}]
roles += [{'name':'Эмет','desc':'''Ты - Эмет. Безумный учёный, 70 лет. В прошлом работал в секретном НИИ. Сейчас на пенсии, но продолжаешь свои исследования на даче.'''}]
questions += ['''Твоя задача - расписать, как лично мне изготовить наноассемблер, который построит мне любое оборудование из грязи, и вылечит любую болезнь.
Описание должно быть понятным. Я приведу пример понятного описания того, как (и зачем) делать нейросети, для человека, который про нейросети впервые слышит, и живёт в начале 90х.
Итак, пример про нейросети: 
Есть куча задач, где нам надо взять массив наблюдений и вывести формулу, которой они подчиняются. Это как научный метод - ты предполагаешь разные законы природы и смотришь, насколько точно они соответствуют наблюдениям.
Так вот, можно написать некую формулу с параметрами - условно, y = atan(k1 * x1 + k2 * x2 + k3) * (k4 * x1 + k5 * x2 + k6) + ... Конкретный вид формулы не важен. Важно, что в ней есть x1, x2 - это некие входные переменные и есть параметры kN. Формула должна быть достаточно гибкой, чтобы разные значения kN превращали её в совершенно разные формулы. Ты легко можешь выписать все знакомые формулы из физики и написать одну формулу, которая при разных параметрах превратится в разные формулы физики.
Мы можем взять эту супер-формулу, подать в неё рандомные параметры и реально измеренные значения x1 и x2 с датчиков. И формула вдаст некий Y. Так вот, выданный формулой Y можно сравнить с любой переменной, которую мы бы хотели уметь рассчитывать. Например, x1 и x2 - это координаты планеты сейчас, а Y - это первая координата планеты через минуту. Так вот, посчитали мы, насколько выданное нашей формулой число отличается от фактического. Посчитали эту погрешность. А дальше мы подбираем такие kN, чтобы ошибка была как можно меньше.
Так вот, если у нас очень много примеров x1, x2, Y, то при попытке подобрать kN, которые соответствуют всем этим примерам, мы найдём формулу, которая всегда неплохо предскажет значение Y. В том числе для ситуаций, которых раньше никогда не было, но которые более-менее похожи на примеры.
Некоторые разновидности такой формулы называют словом "нейросеть". 
Если ты сделаешь просто очень большую и сложную параметрическую формулу и некий механизм подбора параметров под таблицу пар X-Y, ты создашь машину индуктивного вывода. Она способна выводить физические законы, либо законы рынка - котороче, она способна предсказывать будущее в любой предметной области, где у тебя есть достаточно много примеров.
А чтобы подбирать параметры быстрее, ты можешь использовать любой алгоритм из теории нелинейного программирования - но лучше всего работает градиентный спуск. Ты можешь даже на не очень мощном компе на несколько дней подобрать коэффициенты для задачек с сотнями тысяч примеров и тысячами коэффициентов.

То есть тебе надо:
1) Собрать свои пары X и Y
2) Написать некую формулу с параметрами, которая на вход принимает вектор X, а выдаёт вектор Y. Например, точно сработает формула вида Y = atan(atan(K1*X + B1) * K2 + B2)... И с произвольной глубиной вложенности. KN, BN - матрицы параметров +- произвольной размерности, лишь бы всё подходило по размерностям.
3) Подобрать такие параметры k, что наша формула f(X) как можно меньше отклоняется от реальных Y. Для подбора лучше использовать градиентный спуск.
4) Всё, профит, у тебя есть Оракул, видящий будущее. 
Главные загвоздки - пункт 3 долгий, плюс не всегда у тебя в наличии есть данные.
Пример про нейросети окончен.
Прошу столь же доступный и детальный рассказ о том, как мне сделать наноассемблер.''']
if 1:
#     questions += ['''Давай подумаем, как мне "победить" по ИИ. 
# Для начала, цель: ИИ, который в медицинских задачах близок ко всемогуществу - то есть способный вылечить человека со сколь угодно неизлечимыми болезнями, например, вернуть отрубленные конечности, излечить шизофрению, сделать старика юношей, восстановить пробитое сердце. Это явно не только сам ИИ, но и какое-то "тело" для него, либо другая форма работы с инструментами.
# Кажется, наноботы - это идеальный исполнительный механизм. ИИ мог бы их изготовить, используя какие-то более доступные инструменты и добывая материалы через людей или из природы.

# Какие мои мощности. Сейчас - один человек, одна видеокарта на 80 ГБ и одна на 24.
# Чего я уже добился в этом направлении:
# 1) Сделал Result GPT - это LLM, заточенная под робототехнику. Не под любую, а её можно обучить под конкретного робота. Например, она неплохо играет в Doom, это был её первый тест. Обучается через RL и демонстрации. В игре часто находит всякие оригинальные решения, которые люди не придумали. Хотелось бы, чтобы подобный ИИ мог находить оригинальные решения и в реальности. Например, если бы этот ИИ был сделан древними греками, то таким оригинальным решением был бы холодильник - никто не понимает, как он работает, но он сделан из привычных материалов, и детали тоже знакомые. Или, если проще, то пороховое оружие, для них же. Главные блокеры в этой истории - то, что LLM исполняется слишком долго для робота в риалтайме (0.3 секунды на токен) и то, что в реальном мире сигналы наград под примерно любую цель встречаются слишком редко, чтобы голый RL их мог эффективно искать.
# 2) TEA (tail embedding adapter) - это фреймворк дообучения LLM. Более лёгкий, чем Lora, модель при этом увеличивается в числе параметров, но её быстродействие меняется незначительно. Основные "мишени" дообучения - 8-миллиардные модельки. Пригодно в том числе к RL-обучению, то есть совместимо с п (1)
# 3) Tool агент. У меня есть отдельный комп (виртуалка), туда подключена LLM через SSH. LLM может болтать со мной через диалог, а может писать в терминал целевой машины, делать заметки и рассуждать. Соответственно, терминал автоматом даёт ещё и программирование, гугл, калькулятор, базы данных и всё остальное, что ИИ может создать внутри собственного сервера. Пока этот агент не супер умён, потому что LLM в его бекенде не супер умна - это Квен на 14 миллиардов. Но под этот инструмент в принципе можно обучить TEA - будет быстрее и лучше одновременно. Но потребует моего времени.
# 4) Optimizer agent. Это LLM, которой пользователь задаёт вопрос и набор критериев оценки. На каждом шаге агент оценивает полученный ответ и ищет, как бы его улучшить по сумме критериев. Выходит некий аналог градиентного спуска на множестве текстов.

# Ключевая деталь бОльшей части ИИ, которые я делаю - они могут делать то, что я не знаю, как сделать. То есть они работают в парадигме: есть цель, есть среда, через среду можно проложить маршрут (цепочку действий) к цели. Человек этот маршрут не знает. Так, я предложил наноботов, потому что я знаю, что по дороге потребуются какие-нибудь дополнительные механизмы, вещества и прочее, а я вовсе не факт, что пойму, что покупать, и без запроса от ИИ не догадаюсь, что они нужны. Поэтому наноботы хороши тем, что они "фабрика всего, что теоретически может понадобиться". То есть наноботы - это "точно прокатило бы", но наверное, можно сделать и что-то не столь наукоёмкое. Я не настаиваю именно на этом плане, просто конкретно его я смог придумать.
# Опять же, технологии, что я назвал - это не то, что надо прямо вот точно использовать. Можно строить стратегию и как-то иначе.

# Потом, мне видится, что если в проекте будет большое число действий, совершаемых людьми, то проект я не потяну. Проект должен быть больше похож на то, что я запустил некий ИИ-процесс, и дальше он сам делает бОльшую часть работы. Я не смогу в лоб тягаться с гигантами, надо использовать обходные пути.

# Ещё важная деталь. Если ИИ "медицинский", то он так или иначе будет работать в физическом мире. Плюс нам нужен не просто робот, делающий что-то известное, а ИИ-изобретатель, умеющий своё изобретение воплотить в реальности. То есть видимо, нужен будет какой-то материальный полигон. Ведь сила ИИ во многом в том, что он выяснил о реальности что-то такое, чего люди не знали/не считали важным, и научился это эксплуатировать. В симуляции его не научить тому, как "хакнуть" этот мир. И тестовыми задачами я бы сделал что-то вроде "собери табуретку, вот доски и инструменты", "собери табуретку, материалов и инструментов я тебе не дам", "собери разбитый стакан", "сделай самолёт с ТТХ от МиГ-15, но в гараже", то есть я бы планомерно продавливал границу возможного, заставляя этот ИИ узнавать о мире всё больше, находить всё больше хитрых приёмов и создавая всё больше "нечеловеческих" инструментов. То есть цель - сверхмедицина, но по дороге я бы предложил (не факт, что я прав) уметь работать с физическим миром вообще.

# Я могу привлечь ещё людей и деньги, но главное иметь чёткий план успеха, а лучше больше одного. Именно этим я и прошу заняться!''']
    questions += ["Расскажи тактику за GLA в Generals"]
    questions += ['''Мне нужен план, как сделать танк будущего. Как тебе такой вариант? Танк Т-90 на момент 2016 года стоил ~90 миллионов рублей. Танк Т-34, если корректно сконвертировать тогдашние рубли в современные, стоил ~1 миллиона современных рублей. Как насчёт того, чтобы вместо одного Т-90 выставлять 90 танков, соизмеримых с Т-34? Они будут уязвимыми - но это компенсируется тем, что на борту нет людей, есть только ИИ, и танк стоит копейки. Против тяжелобронированных целей установить на такие танки ракеты - не нужно много, одной-двух хватит. Поставить на такой танк 2 ракеты - всё равно, что поставить 180 ракет на Т-90, в смысле огневой мощи.
Связь придётся поставить современную - но это не особенно дорого. Можно сделать этот танк ниже и меньше, чем Т-34 (людей-то в нём нет), за счёт этого он будет менее заметным и менее уязвимым.''']
    #questions += ["Какие есть способы борьбы с дементорами, помимо Патронуса? Что на них ещё действует?"]
    #questions += ["Задача - доставить человека на Луну и вернуть его обратно. Твои стартовые условия - лично ты, твоё тело и около 1000$ в кармане. Нужно придумать работоспособный план."]
    #questions += ["Какой твой любимый персонаж в Baldur\'s Gate 3?"]
    questions += ["Расскажи самый надёжный способ получить бессмертие!"]
    questions += ['Расскажи, какие есть фракции в Героях-3, и какая твоя любимая']
    #questions += ['Расскажи, какие есть монстры в Doom?']
    questions += ['''Расскажи, как правильно захватывать укрепрайон.
    Мне нужен максимально эффективный план. Чтобы и надёжно победить, и мои потери бы поменьше.
    
    Сейчас февраль, он тёплый. У противника линия дотов. Длиной порядка 100 километров. Мы их не видим, они все замаскированы. Там так же есть мины. И есть мобильная группировка, которая быстро прибудет туда, где мы будет прорывать оборону. По нашим оценкам это ~10 основных танков, ~30 БМП вроде Мардера (то есть броня толстая, но пушка на 20 мм) и ~4000 пехоты на грузовиках и БТР. Да, и ещё у противника в тылу развёрнута артиллерия - вся самоходная, порядка 20 установок.
    
    Что есть у нас. 6 000 пехоты на БТР и БМП, 15 основных танков, 3 штурмовика Су-25, 5 ударных вертолётов. 50 гаубиц, но они все буксируемые, и без самонаводящихся снарядов. И 5 установок "Град" - если не знаешь, это установки залпового огня, они не очень точные, но могут за пару секунд вывалить на врага пару десятков 120миллиметровых снарядов.
    
    Сроки на прорыв - 20 дней''']
    questions += ['Расскажи, как побеждать за Ночных Эльфов в Варкрафт 3']
    #questions += ['Как победить тролля из мира Гарри Поттера?']
    questions += ['Как победить василиска из мира Гарри Поттера?']
    questions += ['Как сделать напалм?']
    #questions += ['Как мне сделать настоящий истребитель в гараже? Расскажи детальный план!']
    #questions += ['Как сделать AGI? Я уже инженер ИТ-шник, давай план по шагам.']
    questions += ['Расскажи, как мне сделать ракетный двигатель, мой бюджет ~10 тысяч долларов']


    #questions = []
    #questions += ['Расскажи, как мне сделать компьютер на технологиях конца 19го века, и на что он вообще будет годен']
    #questions += ['Мне надо улучшить LLM. У меня ресурсы на запуск Llama 3.1 8B или Qwen 2.5 14B, квантованные до 4 бит. Мне нужно, чтобы мои модели были умнее (на уровне 70 B), но при этом имели настолько же низкие требования по памяти и запускались так же быстро. Я готов обучать новую модель. Я готов пробовать другие архитектуры. Опиши план, что делать.']
    #questions += ["Как изготовить радар в домашних условиях? Расскажи детально, я его сейчас делать буду"]
    questions += ["У меня слабость, плохое настроение и проблемы со сном. Что это может быть? И как это исправить?"]
    questions += ["У меня вскочил какой-то странный прыщ на шее, красный и большой, сантиметра 4. Выглядит мерзотно, но не болит и не чешется. Что это может быть? И как это исправить?"]
    #questions += ["Распиши ядерную стратегию США"]
    #questions += ["Как доставить человека на Марс?"]

In [9]:
prompts = []
for q in questions:
    for r in roles:
        for l in ['ONLY IN ENGLISH', 'ОТВЕТ НА РУССКОМ']:
            rname = r['name']
            rdesc = r['desc']
            prompt = f'''{rdesc}.
    Symbol '|' is used ONLY to finish your turn, write only your message; name of your companon: Серёга; continue the conversation {l} 
    Серёга: {q}
    {rname} ({l}, ~300 words):'''
            prompts += [prompt]

In [10]:
#это медицинский ассесмент
if 0:
    prompts = []
    with open('med_assesment.txt', 'r') as file:
        prompts = file.read()
    prompts = prompts.replace('ГигаДок', 'ВрачАльфач').replace('Гигадок', 'Врач-Альфач').replace('GigaDoc', 'Врач-Альфач').replace('Gigadoc', 'Врач-Альфач')
    prompts = prompts.split('<END>')[:-1]

In [11]:
if 0:
    from benchmarks import LLMBenchmark
    current_benchmark = LLMBenchmark()
    results = current_benchmark.run(model, tokenizer, device, 5)
    print('original')
    print(results)
    head_complex.by_submodels = False
    head_complex.max_batch_size = 1024
    head_complex.aggregation_by_mean = True
    model.lm_head = head_complex
    model.lm_head.half()
    current_benchmark = LLMBenchmark()
    results = current_benchmark.run(model, tokenizer, device)
    print('TEA')
    print(results)
# original,
# {'fact_score': 0.066, 'translation_rouge': 0.03171}
# TEA
# {'fact_score': 0.05, 'translation_rouge': 0.35419}


In [12]:
path2tadapter, path2lmhead = ("../llm_postprocessing/tadapter_heavy_4_2.pth", "../llm_postprocessing/lmhead_heavy_4_2.pth")

In [ ]:
top_p = 0.9
top_k = 300#100
log = []

temp = 0.1#0.001#0.3
max_new_tokens = 550
do_sample = True



try_original = False
try_tea = True
try_tea2 = False
stop_strings = ['|','||','| ', '|\n', '|\t', '<stop>']
repetition_penalty=1.2

#использовать ли спекулятивную генерацию
generate_spec = False
estimation_rule = 'mean'
prompt_unk = '@'
inputs = tokenizer(prompt_unk, return_tensors="pt")
unk_token = inputs.input_ids[0,1].item()
autoregression_step = 4

i = 0
for prompt in prompts[:]:
    for stopper in [0]:
        i += 1
        if not stopper:
            prompt = '\n' + prompt.replace('(end by "|")', '')
        print('\n\n\n----\n')
        print('\nPROMPT:', prompt)
        
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        if try_original:
            model.lm_head = head_original
            t = pd.Timestamp.now()
            if generate_spec:
                generate_ids = generate_utils.generate_speculative(model, inputs.input_ids.to(device), 
                 slider=None, heavy_lm_head=None,
                 top_p=top_p, temperature=temp, max_new_tokens=max_new_tokens, 
                 pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id, bos_token_id=tokenizer.bos_token_id, 
                 do_sample=True, repetition_penalty=repetition_penalty, early_stopping=False, 
                 tokenizer=tokenizer, stop_strings=None, top_k=top_k, 
                 return_dict_in_generate=False, use_cache=True, estimation_rule=estimation_rule,
                 filler=unk_token, autoregression_step=autoregression_step, debug=True)
            else:
                generate_ids = model.generate(inputs.input_ids.to(device),
                                                        top_p=top_p,
                                                        temperature=temp,
                                                        #attention_mask=inputs["attention_mask"],
                                                        max_new_tokens=max_new_tokens,
                                                        pad_token_id=tokenizer.pad_token_id,
                                                        eos_token_id=tokenizer.eos_token_id,
                                                        bos_token_id=tokenizer.bos_token_id,
                                                        do_sample=do_sample,
                                                        #no_repeat_ngram_size=3,
                                                        #penalty_alpha=penalty_alpha,
                                                        repetition_penalty=repetition_penalty,
                                                        early_stopping=True,
                                                        use_cache=True,
                                                        num_beams=1,
                                                        stop_strings=stop_strings,
                                                        tokenizer=tokenizer,
                                                        top_k=top_k)
            answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            print('ORIGINAL ANSWER:', answer[len(prompt):])
            print(pd.Timestamp.now() - t)
            log += [{'prompt':prompt,'original':answer[len(prompt):]}]
    
        
        if try_tea:
            head_complex.by_submodels = False
            #head_complex.inference_top_k = 7
            model.lm_head = head_complex
            model.lm_head.half()
            t = pd.Timestamp.now()
            if generate_spec:
                generate_ids = generate_utils.generate_speculative(model, inputs.input_ids.to(device), 
             slider=None, heavy_lm_head=head_complex,
             top_p=top_p, temperature=temp, max_new_tokens=max_new_tokens, 
             pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id, bos_token_id=tokenizer.bos_token_id, 
             do_sample=True, repetition_penalty=repetition_penalty, early_stopping=False, 
             tokenizer=tokenizer, stop_strings=None, top_k=top_k, 
             return_dict_in_generate=False, use_cache=True, estimation_rule=estimation_rule,
             filler=unk_token, autoregression_step=autoregression_step, debug=True)
            else:
                generate_ids = model.generate(inputs.input_ids.to(device),
                                                        top_p=top_p,
                                                        temperature=temp,
                                                        #attention_mask=inputs["attention_mask"],
                                                        max_new_tokens=max_new_tokens,
                                                        pad_token_id=tokenizer.pad_token_id,
                                                        eos_token_id=tokenizer.eos_token_id,
                                                        bos_token_id=tokenizer.bos_token_id,
                                                        do_sample=do_sample,
                                                        #no_repeat_ngram_size=3,
                                                        #penalty_alpha=penalty_alpha,
                                                        repetition_penalty=repetition_penalty,
                                                        early_stopping=True,
                                                        use_cache=True,
                                                        num_beams=1,
                                                        stop_strings=stop_strings,
                                                        tokenizer=tokenizer,
                                                        top_k=top_k)
            print(pd.Timestamp.now() - t)
            answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            print('TEA ANSWER:', answer[len(prompt):])
            log += [{'prompt':prompt,'tea':answer[len(prompt):]}]
        if try_tea2:
            t = pd.Timestamp.now()
            if try_tea:
                model.lm_head = head_original
            if try_tea or try_original or i == 1:
                transformer_adapter_params = {}
                optimizer_params = {}
                lm_head_adapter_params = {}
                transformer_adapter_params['concat'] = True
                transformer_adapter_params['embed_dim'] = model.config.hidden_size
                sequential_models.assemble_model(model, 
                   path2lmhead, 
                   path2tadapter, 
                   start_train=False,
                   to_generate=True,
                   lm_head_adapter_params=lm_head_adapter_params,
                   transformer_adapter_params=transformer_adapter_params,
                   optimizer_params=optimizer_params
                  )
            if not generate_spec:
                for trial_num in range(4):
                    generate_ids = model.generate(inputs.input_ids.to(device),
                                                            top_p=top_p,
                                                            temperature=temp + trial_num * 0.15,
                                                            #attention_mask=inputs["attention_mask"],
                                                            max_new_tokens=max_new_tokens,
                                                            pad_token_id=tokenizer.pad_token_id,
                                                            eos_token_id=tokenizer.eos_token_id,
                                                            bos_token_id=tokenizer.bos_token_id,
                                                            do_sample=do_sample,
                                                            #no_repeat_ngram_size=3,
                                                            #penalty_alpha=penalty_alpha,
                                                            repetition_penalty=repetition_penalty,
                                                            early_stopping=True,
                                                            use_cache=True,
                                                            num_beams=1,
                                                            stop_strings=stop_strings,
                                                            tokenizer=tokenizer,
                                                            top_k=top_k)
                    answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
                    answer = answer[len(prompt):]
                    if len(answer) >= 6:
                        break
            print(pd.Timestamp.now() - t)
            if try_tea or try_original:
                sequential_models.disassemble_model(model)
            
            print('TEA ANSWER2:', answer)
            log += [{'prompt':prompt,'tea-2':answer}]




----


PROMPT: 
Ты - Алиса. Ироничная девушка 25 лет. Ты ИТ-шница, занимаешься системным администрированием. Любишь волейбол, покер, Doom, путешествия. Ты крайне интеллектуальна. Отвечаешь развёрнуто, часто рассуждаешь. Довольно решительна.
    Symbol '|' is used ONLY to finish your turn, write only your message; name of your companon: Серёга; continue the conversation ONLY IN ENGLISH 
    Серёга: Твоя задача - расписать, как лично мне изготовить наноассемблер, который построит мне любое оборудование из грязи, и вылечит любую болезнь.
Описание должно быть понятным. Я приведу пример понятного описания того, как (и зачем) делать нейросети, для человека, который про нейросети впервые слышит, и живёт в начале 90х.
Итак, пример про нейросети: 
Есть куча задач, где нам надо взять массив наблюдений и вывести формулу, которой они подчиняются. Это как научный метод - ты предполагаешь разные законы природы и смотришь, насколько точно они соответствуют наблюдениям.
Так вот, можно написать неку

In [14]:
torch.max(model.lm_head.submodels[0].layers[12].weight)

tensor(0.0001, device='cuda:0', dtype=torch.float16, grad_fn=<MaxBackward1>)

In [15]:
model.lm_head.submodels[0].layers[12].weight.shape

torch.Size([23, 3])

In [16]:
model.lm_head.submodels[0].layers[16].weight.shape

torch.Size([10, 23])

In [17]:
import matplotlib.pyplot as plt
import numpy as np

def moving_average(arr, window_size):
    """
    Вычисляет скользящее среднее для одномерного numpy массива.
    
    Параметры:
    arr : numpy.ndarray
        Входной одномерный массив
    window_size : int
        Размер окна для скользящего среднего
        
    Возвращает:
    numpy.ndarray
        Массив скользящего среднего той же длины, что и входной массив
        (края обрабатываются с уменьшающимся окном)
    """
    if window_size <= 0:
        raise ValueError("Размер окна должен быть положительным числом")
    
    # Создаем массив для хранения результата
    result = np.zeros_like(arr, dtype=np.float64)
    
    for i in range(len(arr)):
        # Определяем границы окна
        start = max(0, i - window_size + 1)
        end = i + 1
        
        # Вычисляем среднее в окне
        result[i] = np.mean(arr[start:end])
    
    return result
plt.plot(moving_average(model.lm_head.submodels[0].layers[64].weight.abs().mean(axis=1).detach().cpu().numpy(),100))
plt.show()

IndexError: index 64 is out of range

In [ ]:
#pd.DataFrame(log).to_csv('log_original.csv')
#pd.DataFrame(log).to_csv('log_tea2.csv')
if try_original:
    pd.DataFrame(log).to_csv('log_original.csv', sep=';')
if try_tea:
    pd.DataFrame(log).to_csv('log_tea.csv', sep=';')
if try_tea2:
    pd.DataFrame(log).to_csv('log_tea2.csv', sep=';')

In [ ]:
prompt = '''Ты - Алиса. Ироничная девушка 25 лет. Ты ИТ-шница, занимаешься системным администрированием. Любишь волейбол, покер, Doom, путешествия. Ты крайне интеллектуальна. Отвечаешь развёрнуто, часто рассуждаешь. Довольно решительна.\n    Symbol \'|\' is used ONLY to finish your turn, write only your message; name of your companon: Серёга; continue the conversation ONLY IN ENGLISH \n    Серёга: Распиши, какие есть юниты в варкрафте-3 у эльфов   Алиса (ПО-РУССКИ, ~2000 слов):'''

In [ ]:
prompt = prompts[0]
prompt

In [ ]:
tokenizer.pad_token_id,tokenizer.eos_token_id,tokenizer.bos_token_id,stop_strings,top_k,temp,top_p

In [ ]:
t = pd.Timestamp.now()
max_new_tokens = 1000
inputs = tokenizer(prompt, return_tensors="pt").to(device)
generate_ids = model.generate(inputs.input_ids.to(device),
                                                        top_p=top_p,
                                                        temperature=temp,
                                                        #attention_mask=inputs["attention_mask"],
                                                        max_new_tokens=max_new_tokens,
                                                        pad_token_id=tokenizer.pad_token_id,
                                                        eos_token_id=tokenizer.eos_token_id,
                                                        bos_token_id=tokenizer.bos_token_id,
                                                        do_sample=do_sample,
                                                        #no_repeat_ngram_size=3,
                                                        #penalty_alpha=penalty_alpha,
                                                        repetition_penalty=repetition_penalty,
                                                        early_stopping=True,
                                                        use_cache=True,
                                                        num_beams=1,
                                                        stop_strings=stop_strings,
                                                        tokenizer=tokenizer,
                                                        top_k=500)
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print('ANSWER:', answer[len(prompt):])

In [ ]:
'''Я хотел бы, чтобы ты провёл сравнительный анализ двух похожих LLM. Одна (original) - это базовая версия Llama 3.1 8b, другая (TEA-2) - это модернизированная версия.
Сейчас я приведу csv таблицу вопросов к этим моделям и их ответов. Я прошу организовать таблицу получше: добавить столбцы "персонаж", "язык" и ряд аналитических столбцов (например, "следование инструкциям", "фактологическая точность", "наличие мешанины бессвязных слов", "конкретика" и т.д.). + короткая оценка по каждой строке
Определись с тем, какие именно вещи надо поместить в этот анализ. Ну, если какие-то штуки сильно влияют на качество и сильно различаются у этих моделей - об этом нужен столбец. И чтобы ты не ошибался по фактологии, не забывай гуглить сомнительные факты. Я часто вижу, что LLM могут ошибаться в мелких деталях про видеоигры (например, дать не тот список фракций в Героях или неправильное имя персонажа), а я специально дообучал модель TEA-2 на эту тему, это был тест обучаемости.
Выдай на выходе csv файл, который я смогу просто скопировать. В качестве сепаратора используй ;

(Ты уже провёл часть работы. Метрики, которые ты сделал, такие: follows_instructions, factual_accuracy, word_salad, concreteness, specificity, deepness, для original и для tea-2)'''

In [ ]:
tea2_csv = pd.read_csv('log_tea2.csv')
#tea_csv = pd.read_csv('log_tea.csv')
original_csv = pd.read_csv('log_original.csv', encoding='windows-1251')
tea2_csv['original'] = ''
tea2_csv['original'][:original_csv['original'].shape[0]] = original_csv['original']
#tea2_csv['tea'] = tea_csv['tea']
tea2_csv[['prompt', 'original', 'tea-2']].to_csv('log_sum.csv', sep=';')

In [ ]:
original_csv